# Tag 5

First, let's have a look at our test file

In [18]:
import re

with open("test_input5.txt") as fin:
  lines = list(fin)

print("\n".join(lines))

seeds: 79 14 55 13



seed-to-soil map:

50 98 2

52 50 48



soil-to-fertilizer map:

0 15 37

37 52 2

39 0 15



fertilizer-to-water map:

49 53 8

0 11 42

42 0 7

57 7 4



water-to-light map:

88 18 7

18 25 70



light-to-temperature map:

45 77 23

81 45 19

68 64 13



temperature-to-humidity map:

0 69 1

1 0 69



humidity-to-location map:

60 56 37

56 93 4


The fistst line contains a number of integers representing seeds. We need to extract those. We start by taking the first line and spliting it by its spaces.

In [19]:
seeds = lines[0].split(" ")[1:]

print("Seed strings:", seeds)
seeds = [int(s) for s in seeds]

Seed strings: ['79', '14', '55', '13\n']


Now we iterate the remainder of the file which contains of a number of blocks that follow the pattern

```
[map-source]-to-[map-target] map:
[target] [source] [range]
[target] [source] [range]
```

In [20]:
blocks = "".join(lines[1:]).split("\n\n")
print("Messy blocks:", [blocks[1]])

blocks = [block.strip().split("\n") for block in blocks]
print("Better blocks:", blocks[0])

Messy blocks: ['soil-to-fertilizer map:\n0 15 37\n37 52 2\n39 0 15']
Better blocks: ['seed-to-soil map:', '50 98 2', '52 50 48']


An explaination for this particular regex can be found here: https://regex101.com/r/bcKZ8f/1

In [21]:
def header_to_map(line):
  header_regex = r"^(?P<source>[^-]+)-to-(?P<target>[^-]+) map"
  match = re.match(header_regex, line)
  return (match.group("source"), match.group("target"))

print(header_to_map("seed-to-soil map:"))

('seed', 'soil')


The ranges are representet as homogenous triples - we can just parse them as-is. For convenience, we switch source and target of these mappings.

In [22]:
def ranges_to_mappings(line):
  target_start, source_start, mapping_range = map(int, line.split(" "))
  return source_start, target_start, mapping_range

print(ranges_to_mappings("50 98 2"))
print(ranges_to_mappings("52 50 48"))

(98, 50, 2)
(50, 52, 48)


Next, we combine the previous two functions to create a new mapping
$kind_{from} \rightarrow (kind_{to}, ranges)$

In [23]:
def translate_block_to_mapping(block):
  # Parse the first line of the block as mapping header
  mapping_pair = header_to_map(block[0])

  # parse the remainder as mapping pairs k -> v
  mappings = [ranges_to_mappings(l) for l in block[1:]]
  return mapping_pair[0], (mapping_pair[1], mappings)

print("Block:", blocks[0])
print("Translated block:", translate_block_to_mapping(blocks[0]))

mappings = dict([translate_block_to_mapping(b) for b in blocks])
print(mappings)

Block: ['seed-to-soil map:', '50 98 2', '52 50 48']
Translated block: ('seed', ('soil', [(98, 50, 2), (50, 52, 48)]))


Here comes the tricky part. A straight-forward solution does not work with this problem, as the data tends to become huge. Therefore, we need a more elaborate strategy. In fact, we will retain the strange list representation that this task introduced.

The idea is as follows: Any given mappings applies to ranges of numbers (in part 1, these ranges are only single integers). Therefore, we need to only map those numbers, that are affected by this mapping. The problem: There are different kinds of interactions between ranges. More precisely, we are interested in three parts of our range:

1. The part that lies before the mapping
2. The part that oberlaps with the mapping
3. The part that lies after the mapping

An example: A range $A=(a, r_a)=(3,5)$ represents the list of 5 integers that starts at 3, i.e. $[3,4,5,6,7]$ (the functions `unfold_interval`and `print_test` exist solely to transfer the results into a more readable form).

Now imagine that there is a second range $B=(b,r_b)$. There are 6 different scenarios for depending on its exact location:

* $B$ strictly before $A$:
  * e.g. $B=(1,1)$
  * Parts of A before B: None
  * Parts of A within B: None
  * Parts of A after B: $[3,4,5,6,7]$
* $B$ starts before, but ends in $A$
  * e.g. $B=(1,3)$
  * Parts of A before B: None
  * Parts of A within B: $[3]$
  * Parts of A after B: $[4,5,6,7]$
* $B$ starts before, but ends after $A$:
  * e.g. $B=(1,9)$
  * Parts of A before B: None
  * Parts of A within B: $[3,4,5,6,7]$
  * Parts of A after B: None
* $B$ starts within $A$ and ends in $A$:
  * e.g. $B=(4,2)$
  * Parts of A before B: $[3]$
  * Parts of A within B: $[4,5]$
  * Parts of A after B: $[6,7]$
* $B$ starts within $A$ and ends after $A$:
  * e.g. $B=(4,5)$
  * Parts of A before B: $[3]$
  * Parts of A within B: $[4,5,6,7]$
  * Parts of A after B: None
* $B$ starts after $A$.
  * e.g. $B=(8,1)$
  * Parts of A before B: $[3,4,5,6,7]$
  * Parts of A within B: None
  * Parts of A after B: None


In [37]:
def unfold_interval(s):
  if s is None:
    return []
  a, b = s
  return list(range(a,a+b))

def intersect_intervalls(a, b):
  a, ra = a
  b, rb = b
  if b<= a:
    # B starts before A ...
    if b+rb <= a:
      # ... and ends before A
      return None, None, (a, ra)
    else:
      if b+rb < a+ra:
        # ... and ends within A
        return None, (a,rb-(a-b)), (b+rb,a+ra-b-rb)
      else:
        # ... and ends after A
        return None, (a, ra), None
  else:
    if a+ra <= b:
        # B starts after A
        return (a,ra), None, None
    else:
        # B starts within A ...
        if a+ra < b+rb:
          # ... and ends after A
          return (a, b-a), (b, ra-(b-a)), None
        else:
          # ... and ends within A
          return (a, b-a), (b,rb), (b+rb, ra-(b-a)-rb)

def print_test(a,b):
  l, m, r = map(unfold_interval, intersect_intervalls(a,b))
  ua = unfold_interval(a)
  assert l+m+r == ua, "{} != {}".format(l+m+r, ua)
  print(ua, "overlapped with ", unfold_interval(b),)
  print("\tBefore:", l, "Overlap:", m, "After:", r)
  print("")

print_test((3,5), (1,1))
print_test((3,5), (1,3))
print_test((3,5), (1,9))
print_test((3,5), (4,2))
print_test((3,5), (4,5))
print_test((3,5), (8,1))



[3, 4, 5, 6, 7] overlapped with  [1]
	Before: [] Overlap: [] After: [3, 4, 5, 6, 7]

[3, 4, 5, 6, 7] overlapped with  [1, 2, 3]
	Before: [] Overlap: [3] After: [4, 5, 6, 7]

[3, 4, 5, 6, 7] overlapped with  [1, 2, 3, 4, 5, 6, 7, 8, 9]
	Before: [] Overlap: [3, 4, 5, 6, 7] After: []

[3, 4, 5, 6, 7] overlapped with  [4, 5]
	Before: [3] Overlap: [4, 5] After: [6, 7]

[3, 4, 5, 6, 7] overlapped with  [4, 5, 6, 7, 8]
	Before: [3] Overlap: [4, 5, 6, 7] After: []

[3, 4, 5, 6, 7] overlapped with  [8]
	Before: [3, 4, 5, 6, 7] Overlap: [] After: []



And now we need to apply this mappings. This is best explained by example. Let's say we have a given range $A=(3,20) = [3,...,22]$ and mappings (sorted by starting point):
* $B_0= (1,1)\mapsto 99$
* $B_1=(2,3) \mapsto 15$ (maps $[2,3,4]$ to $[15, 16, 17]$)
* $B_2=(10,2) \mapsto 2$. (maps $[10,11]$ to $[2,3]$)
* $B_3=(15,31) \mapsto 2$. (maps $[15,16,...,45]$ to $[2,3,...,32]$)

We start our process with our range.
1. We consider mapping $B_0$ and see that it does not overlap -- skip.  
2. We consider mapping $B_1$ and see that it does overlap with $A$ in $[3,4]$ we therefore map this interval to $[16, 17]=(16,2)$ and store it. There is no left remainder, but the right remainder of $A$ is $A' := (5,18) = [5,...,22]$. We continue with that remainder.
3. We consider mapping $B_2$ and see that it overlaps with $A'$ in $(10,2)=[10,11]$.  The left remainder of $A'$ is $[5,6,7,8,9] = (5,5)$ and can be stored unchanged. We map the overlap interval to $[2, 3]=(2,1)$. The right remainder of $A'$ is $A'' := (12,11) = [12,...,22]$. We continue with that remainder.
4. We consider mapping $B$ and see that is does overlap and end after $A''$. It is, therefore, the last range we have to consider. The left remainder is $(12,3)=[12,13,14]$ and the overlap is $(15,7) = [15,...,22]$. We map this overlap to $(2,7) = [2,...,9]$

Finally, we are done and have these new ranges:
$$(16,2), (5,5), (2,1), (12,3), (2,7)$$
which corresponds to the following numbers:
$$ \{ 2,3,4,5,6,7,8,9,12,13,14,16,17 \} $$

More concretely, the mapping did the following:

|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| $B_1$  |$B_1$||||||$B_2$|$B_2$||||$B_3$|$B_3$|$B_3$|$B_3$|$B_3$|$B_3$|$B_3$|$B_3$|
|16|17|5|6|7|8|9|2|3|12|13|14|2|3|4|5|6|7|8|9

*Note*: We could obviously simplify the ranges to $(2,8)$ and $(12,5)$, but we are lazy and don't do that.

In [27]:
def get_minimal_location(ranges):
  kind = "seed"
  while kind != "location":
    kind, mapping = mappings[kind]
    new_ranges = []
    for a, ra in ranges:
      matched = False
      # We sort mappings my starting point
      for b, target, rb in sorted(mapping, key=lambda x: x[0]):
        # Calculate left remainder (l), overlap (m) and right remainder (r)
        l,m,r = intersect_intervalls((a, ra), (b, rb))
        if m: # Does it overlap?
          matched = True
          if l:
            # If there is a left remainder, store it unchanged
            new_ranges.append(l)
          # Apply the map to the overlap and store it
          mapped_fraction = (target-b+m[0], m[1])
          new_ranges.append(mapped_fraction)
          if r:
              # If there is a right remainder, use this for further checks
              a, ra = r
      if not matched:
        # If nothing matched or we are done,
        new_ranges.append((a,ra))
    ranges = new_ranges

  # The range with the smallest start is our solution.
  values = list(sorted(new_ranges, key=lambda x: x[0]))
  return values[0][0]

In [28]:
seed_pairs = [(s, 1) for s in seeds]
print("The lowest location according to task 1 is:", get_minimal_location(seed_pairs))
seed_pairs = list(zip(seeds[::2], seeds[1::2]))
print("The lowest location according to task 2 is:", get_minimal_location(seed_pairs))

The lowest location according to task 1 is: 35
The lowest location according to task 2 is: 46
